In [ ]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import pickle
import csv
import pandas as pd

In [ ]:
#This script is to translate the grown 7_2 model back to a voxel size of 3_6
########################################################################################
#definition to properly load a value lattice
def lattice_from_csv(file_path):
    # read metadata
    meta_df = pd.read_csv(file_path, nrows=3)

    shape = np.array(meta_df['shape'])
    unit = np.array(meta_df['unit'])
    minbound = np.array(meta_df['minbound'])

    # read lattice
    lattice_df = pd.read_csv(file_path, skiprows=5)

    # create the buffer
    buffer = np.array(lattice_df['value']).reshape(shape)

    # create the lattice
    l = tg.to_lattice(buffer, minbound=minbound, unit=unit)

    return l
########################################################################################


########################################################################################
#Loading the complete lattice and the final grown abm_f
lattice_path = os.path.relpath("data\lattice_complete_3_6.csv")
complete_lattice_3_6 = tg.lattice_from_csv(lattice_path)

lattice_path = os.path.relpath("data\abm\abm_f_354.csv")
avail_lattice = lattice_from_csv(lattice_path)
grow_7_2 = tg.to_lattice(np.copy(avail_lattice), avail_lattice)
########################################################################################


########################################################################################
#Using np.kron to upscale the 7_2 data
grow_3_6_start = np.kron(grow_7_2, np.ones((2, 2, 2)))
grow_3_6_final = grow_3_6_start[:]
grow_3_6_reference = complete_lattice_3_6[:]

#Removing any voxels that are not in the original 3_6 lattice
for i in range(len(grow_3_6_start)):
    for j in range(len(grow_3_6_start[0])):
        for k in range(len(grow_3_6_start[0][0])):
            if grow_3_6_reference[i][j][k] == False:
                grow_3_6_final[i][j][k] = -1

#Exporting the new grown model in 3_6 size
lattice_grow_3_6 = tg.to_lattice(grow_3_6_final, grow_3_6_reference)
csv_path_lattice = os.path.relpath("data\grown_lattice_3_6.csv")
lattice_grow_3_6.to_csv(csv_path_lattice)
########################################################################################
print("done")